# Subqueries

##### Импорт библиотек

In [1]:
import pandas as pd
import numpy as np
import sqlite3

In [2]:
sqlite3.sqlite_version

'3.39.4'

##### Подключение к БД

In [3]:
con = sqlite3.connect('db')

## Colab

In [5]:
! gdown --id 1cUYNwlHjUZ6gZZMSFBdX0NCnvgRTprsQ

df = pd.read_csv('/content/german_credit_augmented.csv')

"gdown" ­Ґ пў«пҐвбп ў­гваҐ­­Ґ© Ё«Ё ў­Ґи­Ґ©
Є®¬ ­¤®©, ЁбЇ®«­пҐ¬®© Їа®Ја ¬¬®© Ё«Ё Ї ЄҐв­л¬ д ©«®¬.


FileNotFoundError: [Errno 2] No such file or directory: '/content/german_credit_augmented.csv'

## Pycharm

In [4]:
df = pd.read_csv(
    r'C:\PyCharm\Python\SQL_for_data_analysis\2.DB_connect_and_upload\sql-course-main\german_credit_augmented.csv')

In [5]:
df

,age,sex,job,housing,saving_accounts,checking_account,credit_amount,duration,purpose,default,contract_dt,client_id
0,33,male,2,own,NaN,NaN,3074,9,radio/TV,0,2008-06-29 18:52:00,210
1,43,male,1,own,little,little,1344,12,car,0,2007-05-20 18:30:19,929
2,52,male,2,own,quite rich,NaN,936,9,education,0,2008-04-27 08:23:07,200
3,35,female,3,own,little,NaN,1393,11,car,0,2007-05-06 10:58:22,45
4,28,male,2,own,little,NaN,776,12,radio/TV,0,2007-07-21 13:22:14,358
...,...,...,...,...,...,...,...,...,...,...,...,...
995,65,male,2,free,little,little,2600,18,radio/TV,1,2007-12-16 20:17:19,624
996,30,male,3,own,little,moderate,4455,36,business,1,2007-07-12 14:08:58,181
997,33,male,2,own,little,moderate,6403,24,radio/TV,0,2008-04-08 03:24:26,730
998,29,female,2,own,NaN,NaN,5003,21,car,1,2007-11-29 15:51:45,557


In [6]:
df.dtypes

age                  int64
sex                 object
job                  int64
housing             object
saving_accounts     object
checking_account    object
credit_amount        int64
duration             int64
purpose             object
default              int64
contract_dt         object
client_id            int64
dtype: object

In [7]:
df.contract_dt = pd.to_datetime(df.contract_dt, format='%Y-%m-%d %H:%M:%S')

In [8]:
df.dtypes

age                          int64
sex                         object
job                          int64
housing                     object
saving_accounts             object
checking_account            object
credit_amount                int64
duration                     int64
purpose                     object
default                      int64
contract_dt         datetime64[ns]
client_id                    int64
dtype: object

In [9]:
df.to_sql('german_credit', con, index=False, if_exists='replace')

1000

## Подзапросы

In [10]:
t = pd.DataFrame({'id': [1, 1, 2, 2, 3],
                  'name': ['a', 'c', 'b', 'd', 'e']})

In [11]:
t.to_sql('dupl_test', con, index=False, if_exists='replace')

5

In [12]:
def select(sql):
    return pd.read_sql(sql, con)

Посмотрим таблицу дубликатов

In [13]:
sql = '''select * from dupl_test t'''

In [14]:
select(sql)

,id,name
0,1,a
1,1,c
2,2,b
3,2,d
4,3,e


Посмотрим количество имён

In [15]:
sql = '''select t.id, count(*)
from dupl_test t
group by t.id'''

In [16]:
select(sql)

,id,count(*)
0,1,2
1,2,2
2,3,1


Посмотрим только повторяющиеся элементы
Используем Having т.к. функцию агрегации (например count) нельзя засовывать в where

In [17]:
sql = '''select t.id, count(*)
from dupl_test t
group by t.id
having count (*) > 1'''

In [18]:
select(sql)

,id,count(*)
0,1,2
1,2,2


**Показ повторяющихся id хард кодом**

In [21]:
sql = '''select * from dupl_test t
where t.id in(1, 2)'''

In [22]:
select(sql)

,id,name
0,1,a
1,1,c
2,2,b
3,2,d


**Использую подзапрос для показа id, которые повторяются**

In [25]:
sql = '''select * from dupl_test t
where t.id in (select t.id
from dupl_test t
group by t.id
having count (*) > 1)'''

In [26]:
select(sql)

,id,name
0,1,a
1,1,c
2,2,b
3,2,d


**Для избежания ошибок в подзапросах можно строить промежуточные таблицы:**

##### Создаём курсор

In [28]:
cur = con.cursor()

In [29]:
sql = '''
drop table if exists dupls;

create table dupls as
select t.id from dupl_test t
group by t.id
having count(*) > 1'''

In [30]:
cur.executescript(sql)

In [31]:
sql = '''select * from dupls t'''

In [32]:
select(sql)

,id
0,1
1,2


**Теперь вместо подзапроса можно подставлять новую табличку:**

In [33]:
sql = '''select * from dupl_test t
where t.id in dupls'''

In [34]:
select(sql)

,id,name
0,1,a
1,1,c
2,2,b
3,2,d


**Подзапросы позволяют пропустить лишнее действие в создании и дальнейшем хранении промеждуточной таблицы**
**При использовании промежуточной таблицы, в дальнейшем можно будет её удалить, что бы не занимать память**

In [35]:
sql = '''select * from
(select t.id, count(*) as cnt from dupl_test t
group by t.id) t
where t.cnt > 1'''

In [36]:
select(sql)

,id,cnt
0,1,2
1,2,2


#### CTE - with (Command table expressions)
Лучше не использовать подзапросы, а использовать CTE
Пример подзапроса ↓

In [38]:
sql = '''
select * from (

select * from
(select t.id, count(*) as cnt from dupl_test t
group by t.id) t

where t.cnt > 1) t

where t.id = 1'''

In [39]:
select(sql)

,id,cnt
0,1,2


In [43]:
sql = '''select t.id, count(*) as cnt from dupl_test t
group by t.id'''

In [44]:
select(sql)

,id,cnt
0,1,2
1,2,2
2,3,1


**Вместо подзапроса можно воспользоваться CTE (with)**

In [51]:
sql = '''
with id_cnt as (
select t.id, count(*) as cnt from dupl_test t
group by t.id)

select * from id_cnt t
where t.cnt > 1'''

In [52]:
select(sql)

,id,cnt
0,1,2
1,2,2


**С помощью оператора with сильно упрощается логика. При такой записи запрос можно читать сверху вниз, а не искать самый глубокий подзапрос**
**Логика данного запроса:**
1. Создаётся первая таблица, в который есть count
2. Создаётся вторая таблица, которая основана на 1-ой и она фильтрует с помощью where
3. Далее выбирается где id = 1

In [57]:
sql = '''
with id_cnt as (
select t.id, count(*) as cnt from dupl_test t
group by t.id),

id_cnt_2 as (
select * from id_cnt t
where t.cnt > 1)

select * from  id_cnt_2 t
where t.id = 1'''

In [58]:
select(sql)

,id,cnt
0,1,2


**Как выглядит плохочитаемый подзапрос по сравнению с CTE ↓**

In [55]:
sql = '''
select * from (

select * from
(select t.id, count(*) as cnt from dupl_test t
group by t.id) t

where t.cnt > 1) t

where t.id = 1'''

In [56]:
select(sql)

,id,cnt
0,1,2


**Ещё 1 пример:**

In [60]:
t = pd.DataFrame({'purpose':['машина','машина','машина','на машину','на покупку машины',
                             'автомобиль','на возвращение 2007',
                             'на свадьбу','свадьба','свадьба','свадьба','для свадьбы',
                             'недвижимость','на покупку недвижимости']})

In [61]:
t.to_sql('purpose', con, index=False, if_exists='replace')

14

In [62]:
cat = '''select t.purpose,

case when t.purpose like '%свадьб%' then 'свадьба'
when t.purpose like '%машин%' or t.purpose like '%авто%' then 'машина'
when t.purpose like '%недвиж%' then 'недвижимость'

else 'другое' end as purpose_cat

 from purpose t'''

select(cat)

,purpose,purpose_cat
0,машина,машина
1,машина,машина
2,машина,машина
3,на машину,машина
4,на покупку машины,машина
5,автомобиль,машина
6,на возвращение 2007,другое
7,на свадьбу,свадьба
8,свадьба,свадьба
9,свадьба,свадьба


In [63]:
sql = f'''select
t.purpose_cat,

count(*)

 from ({cat}) t
 group by t.purpose_cat'''

select(sql)

,purpose_cat,count(*)
0,другое,1
1,машина,6
2,недвижимость,2
3,свадьба,5


**Команда print(sql) выводит ТЕКСТ запроса sql**

In [64]:
print(sql)

select
t.purpose_cat,

count(*)

 from (select t.purpose,

case when t.purpose like '%свадьб%' then 'свадьба'
when t.purpose like '%машин%' or t.purpose like '%авто%' then 'машина'
when t.purpose like '%недвиж%' then 'недвижимость'

else 'другое' end as purpose_cat

 from purpose t) t
 group by t.purpose_cat


In [65]:
sql = '''select t.purpose,

case when t.purpose like '%свадьб%' then 'свадьба'
when t.purpose like '%машин%' or t.purpose like '%авто%' then 'машина'
when t.purpose like '%недвиж%' then 'недвижимость'

else 'другое' end as purpose_cat
from purpose t'''

In [66]:
select(sql)

,purpose,purpose_cat
0,машина,машина
1,машина,машина
2,машина,машина
3,на машину,машина
4,на покупку машины,машина
5,автомобиль,машина
6,на возвращение 2007,другое
7,на свадьбу,свадьба
8,свадьба,свадьба
9,свадьба,свадьба


Теперь сделаем промежуточную таблицу с помощью with

In [73]:
sql = '''
with categories as (
select t.purpose,

case when t.purpose like '%свадьб%' then 'свадьба'
when t.purpose like '%машин%' or t.purpose like '%авто%' then 'машина'
when t.purpose like '%недвиж%' then 'недвижимость'

else 'другое' end as purpose_cat
from purpose t)

select t.purpose_cat,
count(*) from categories t
group by t.purpose_cat'''

In [74]:
select(sql)

,purpose_cat,count(*)
0,другое,1
1,машина,6
2,недвижимость,2
3,свадьба,5


**Иногда лучше просто создать дополнительную таблицу, нежели использовать CTE (with) или подзапросы**

#### Создание новой таблицы для подзапросов

In [76]:
sql = '''
drop table if exists categories;

create table categories as
select t.purpose,

case when t.purpose like '%свадьб%' then 'свадьба'
when t.purpose like '%машин%' or t.purpose like '%авто%' then 'машина'
when t.purpose like '%недвиж%' then 'недвижимость'

else 'другое' end as purpose_cat

 from purpose t'''


**Для создания используем cur**

In [77]:
cur.executescript(sql)

In [79]:
sql = '''select * from categories t'''

In [80]:
select(sql)

,purpose,purpose_cat
0,машина,машина
1,машина,машина
2,машина,машина
3,на машину,машина
4,на покупку машины,машина
5,автомобиль,машина
6,на возвращение 2007,другое
7,на свадьбу,свадьба
8,свадьба,свадьба
9,свадьба,свадьба


**Теперь данная таблица ↑ есть в БД**

In [82]:
print(cat)

select t.purpose,

case when t.purpose like '%свадьб%' then 'свадьба'
when t.purpose like '%машин%' or t.purpose like '%авто%' then 'машина'
when t.purpose like '%недвиж%' then 'недвижимость'

else 'другое' end as purpose_cat

 from purpose t


In [84]:
sql = f'''select
t.purpose_cat,

count(*)

 from categories t
 group by t.purpose_cat'''

In [85]:
select(sql)

,purpose_cat,count(*)
0,другое,1
1,машина,6
2,недвижимость,2
3,свадьба,5


In [86]:
sql = '''select t.purpose, count(*)
from categories t
where t.purpose_cat = 'другое'

group by t.purpose
order by count(*) desc'''

In [88]:
select(sql)

,purpose,count(*)
0,на возвращение 2007,1
